In [ ]:
!uv pip install selenium

In [ ]:
# Import libraries
import os
from openai import OpenAI
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

class Website:
    def __init__(self, url):
        """
        Create this WebsiteSelenium object from the given URL using Selenium and BeautifulSoup.
        """
        self.url = url

        # Set up Selenium WebDriver with headless Chrome
        chrome_options = Options()
        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--disable-dev-shm-usage")
        chrome_options.add_argument("--disable-blink-features=AutomationControlled")  # Prevent detection
        chrome_options.add_argument("--disable-infobars")  # Disable "Chrome is being controlled" infobar
        
        service = Service()  # Use default ChromeDriver path
        driver = webdriver.Chrome(service=service, options=chrome_options)

        try:
            # Fetch the webpage
            driver.get(url)

            # Get the page source
            page_source = driver.page_source

            # Parse the page source with BeautifulSoup
            soup = BeautifulSoup(page_source, 'html.parser')
            self.title = soup.title.string if soup.title else "No title found"
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        finally:
            # Close the WebDriver
            driver.quit()

In [ ]:
# Initialise OpenAI client using Ollama
OLLAMA_BASE_URL = "http://localhost:11434/v1"
ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key='hohoho')
OL_MODEL = "llama3.2:latest"

# Initialise OpenAI client using OpenRouter
OPENROUTER_BASE_URL = "https://openrouter.ai/api/v1"
openrouter_api_key = os.getenv("OPENROUTER_API_KEY")
openrouter = OpenAI(base_url=OPENROUTER_BASE_URL, api_key=openrouter_api_key)
OR_MODEL = 'xiaomi/mimo-v2-flash:free'

In [ ]:
# Choose which way to access LLM
LLM_access = "ollama"

if LLM_access == "ollama":
    myAI = ollama
    MODEL = OL_MODEL
    print("We will use OLLAMA.")
elif LLM_access == "openrouter":
    myAI = openrouter
    MODEL = OR_MODEL
    print("We will use OPENROUTER")
else:
    print("Please set to ollama or openrouter.")

In [ ]:
# Define call_LLM function
def call_LLM(sys_prompt, user_prompt):
    response = myAI.chat.completions.create(
        model= MODEL,
        messages=[
            {"role": "system", "content": sys_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    result = response.choices[0].message.content
    return result

In [ ]:
# Job and profile URLs
job_url = "ENTER JOB PROFILE URL"
profile_url = "ENTER CANDIDATE PROFILE URL - NO LINKEDIN"

# If it is a LinkedIn profile, it can't be scraped due to security, ask the user to find a different profile
if "linkedin" in profile_url:
    print("LinkedIn profiles can't be scraped. Please find a different online profile for the candidate.")

In [ ]:
# Summarise the job profile
job_system_prompt = f"You are a highly experienced recruitment director reviewing a job profile."
job_system_prompt += "\nYou are able to decide what are the most relevant information in this job profile.\n"
job_system_prompt += "Please list out the key aspects of the job profile for candidate assessment.\n"
job_system_prompt += "Here is an example:\n \
    Job title & summary: A title reflecting seniority and a brief overview of the role’s purpose.\n \
    Key responsibilities: A detailed list of daily tasks, expectations, and major contributions. \n \
    Required skills & competencies: technical abilities (e.g., software, tools) \
    and behavioural attributes (e.g., leadership, communication).\n \
    Qualifications & experience: mandatory education, certifications, and previous experience."

job_profile = Website(job_url)
job_user_prompt = f"Here is the job profile:\n"
job_user_prompt += job_profile.text
job_user_prompt += "\nPlease create a succinct and coherent summary of this job profile."

summarised_job = call_LLM(
    sys_prompt= job_system_prompt,
    user_prompt= job_user_prompt
)

In [ ]:
print(summarised_job)

In [ ]:
# Summarise the candidate profile
profile_system_prompt = f"You are a highly experienced recruitment director reviewing a candidate's profile."
profile_system_prompt += "\nYou are able to decide what are the most relevant information to evaluate this candidate.\n"
profile_system_prompt += "Please list out the key aspects of the candidate's profile for assessment.\n"
profile_system_prompt += "Here is an example:\n \
    Job title & summary: The candidate's title reflecting seniority and a brief of their current role.\n \
    Key responsibilities: A detailed list of the candidate's daily tasks, expectations, and major contributions. \n \
    Required skills & competencies: the candidate's technical abilities (e.g., software, tools) \
    and behavioural attributes (e.g., leadership, communication).\n \
    Qualifications & experience: the candidate's education, certifications, and previous experience."

candidate_profile = Website(profile_url)
profile_user_prompt = f"Here is the candidate's profile:\n"
profile_user_prompt += candidate_profile.text
profile_user_prompt += "\nPlease create a succinct and coherent summary of this candidate's profile."

summarised_profile = call_LLM(
    sys_prompt= profile_system_prompt,
    user_prompt= profile_user_prompt
)

In [ ]:
print(summarised_profile)

In [ ]:
# Set system_prompt
system_prompt = "You are a highly experienced headhunter that review potential \
    candidates' profiles. You will create short summaries about the candidate's \
    fit against published job profiles. You will first score the candidate's \
    overall fit against a particular job profile between 0 and 100, \
    with 100 being the perfect fit and 0 being totally incompatible. \
    For example, if the person is significantly under-qualified or \
    over-qualified, this should result in a relatively low score. \
    If the score is higher than 50, you will go on to outline the key \
    strengths of the candidate and key areas to probe during an interview. \
    If the score is lower than 50, you will reply simply that \
    the candidate is unlikely to be a good fit for the job profile. \
    Respond in markdown without code blocks."

In [ ]:
# Define get_user_prompt function with both contents and links
def get_user_prompt(job, profile):
    user_prompt = f"You are looking at a candidate's profile."
    user_prompt += "\nThe contents of the candidate's profile is as follows:\n"
    user_prompt += profile
    user_prompt += f"\nPlease assess the candidate against this job profile."
    user_prompt += "\nThe details of the job profile is as follows:\n"
    user_prompt += job

    user_prompt += "\n Please use all the information to come up with a score \
        of overall fit and a short summary of strengths and areas to probe \
        in markdown without code blocks."

    return user_prompt

In [ ]:
# Test the user prompt comes out correctly
print(get_user_prompt(
    job= summarised_job,
    profile= summarised_profile
    ))

In [ ]:
# Produce review summary
review_summary = call_LLM(
    sys_prompt= system_prompt,
    user_prompt= get_user_prompt(
        job= summarised_job,
        profile= summarised_profile
    )
)
display(Markdown(review_summary))